In [10]:
import os
import json
import pandas as pd

In [2]:
from glob2 import glob
jsonFiles = glob('/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/*/posts/*.json') #Can be used absolute or relative paths
print(jsonFiles)

'/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data'

In [ ]:
dfList = [ ]
for jsonFile in jsonFiles:
    if jsonFile:
        df = json.loads(json_string)
    else:
        df = pd.read_json(jsonFile, orient='records') 
        dfList.append(df)
    
dfTrainingDF = pd.concat(dfList, axis=0)

In [ ]:
df